<a href="https://colab.research.google.com/github/Kiichiro-T/python/blob/main/experimental_design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy import stats as st

from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
print('肥料100,200,300,400 g ＋ 土A,Bの作物の収量')

fertilizer_100 = np.array([14.5,15.1,14.1,16.2,15.3,17.5])
fertilizer_200 = np.array([16.5,16.1,15,18.6,16.9,18.6])
fertilizer_300 = np.array([17.8,19,15.2,21.7,20.5,19.4])
fertilizer_400 = np.array([18.1,20.2,17.2,23.6,24.9,25.5])

data = pd.DataFrame({'100':fertilizer_100,
                     '200':fertilizer_200,
                     '300':fertilizer_300,
                     '400':fertilizer_400})
data.index = ['A', 'A', 'A', 'B', 'B', 'B']
data

肥料100,200,300,400 g ＋ 土A,Bの作物の収量


,100,200,300,400
A,14.5,16.5,17.8,18.1
A,15.1,16.1,19.0,20.2
A,14.1,15.0,15.2,17.2
B,16.2,18.6,21.7,23.6
B,15.3,16.9,20.5,24.9
B,17.5,18.6,19.4,25.5


In [ ]:
y_mean = data.stack().mean()

yij_mean = pd.DataFrame({})
for j in ['100', '200', '300', '400']:
  yij_mean[j] = np.array([data[j]['A'].mean(), data[j]['B'].mean()])
yij_mean.index = ['A', 'B']

yi_mean = {
  'A': data.loc['A'].stack().mean(),
  'B': data.loc['B'].stack().mean()
}

yj_mean = {
  '100': data['100'].mean(),
  '200': data['200'].mean(),
  '300': data['300'].mean(),
  '400': data['400'].mean()
}

# 総平方和
St = 0
# 土の種類の平方和（水準1）
S1 = 0
# 肥料の量の平方和（水準2）
S2 = 0
# 交互作用平方和
S12 = 0
# 誤差平方和
Sr = 0
for j in ['100', '200', '300', '400']:
  for i in ['A', 'B']:
    for d in data[j][i]:
      St += (d - y_mean)**2
      S1 += (yi_mean[i] - y_mean)**2
      S2 += (yj_mean[j] - y_mean)**2
      S12 += (yij_mean[j][i] - yi_mean[i] - yj_mean[j] + y_mean)**2
      Sr += (d - yij_mean[j][i])**2

# 全体
dft = data.size - 1
# 土の種類
df1 = 2 - 1
V1 = S1 / df1
# 肥料の量
df2 = 4 - 1
V2 = S2 / df2
# 交互作用
df12 = df1 * df2
V12 = S12 / df12
# 残差
dfr = dft - df1 - df2 - df12
Vr = Sr / dfr

# F値
F1 = V1 / Vr
F2 = V2 / Vr
F12 = V12 / Vr

S = np.array([S1, S2, S12, Sr, St])
DF = np.array([df1, df2, df12, dfr, dft])
V = np.array([V1, V2, V12, Vr, None])
F = np.array([F1, F2, F12, None, None])

anova = pd.DataFrame({'平方和':S,
                     '自由度':DF,
                     '平均平方':V,
                     'F値':F})
anova.index = ['土の種類', '肥料の量', '交互作用', '残差', '全体']
anova

,平方和,自由度,平均平方,F値
土の種類,66.333750,1,66.3338,46.3332
肥料の量,126.637917,3,42.2126,29.485
交互作用,17.791250,3,5.93042,4.14232
残差,22.906667,16,1.43167,None
全体,233.669583,23,None,None


In [ ]:
print('2元配置分散分析を次のようにモデル化する')
print('Yijk = µ + αi + βj + γij + εijk')

# 交互作用に関する仮説
print('H_0: γij = 0 がすべてのi,jについて成り立つ')
print('H_1: γij ≠ 0 となるiが存在する')
print('α = 0.05')

f12_a = sp.stats.f.ppf(0.95, df12, dfr)
print(F12, f12_a)
if F12 > f12_a:
  print('帰無仮説を棄却する')
else:
  print('帰無仮説を棄却しない')

print('交互作用は有意である')

# 土の種類（水準1）に関する仮説
print('H_0: αi  = 0 がすべてのi,jについて成り立つ')
print('H_1: αi  ≠ 0 となるiが存在する')
print('α = 0.05')

f1_a = sp.stats.f.ppf(0.95, df1, dfr)
print(F1, f1_a)
if F1 > f1_a:
  print('帰無仮説を棄却する')
else:
  print('帰無仮説を棄却しない')

print('土の種類は有意である')

# 肥料の量（水準2）に関する仮説
print('H_0: βj  = 0 がすべてのi,jについて成り立つ')
print('H_1: βj  ≠ 0 となるiが存在する')
print('α = 0.05')

f2_a = sp.stats.f.ppf(0.95, df2, dfr)
print(F1, f2_a)
if F1 > f2_a:
  print('帰無仮説を棄却する')
else:
  print('帰無仮説を棄却しない')

print('肥料の量は有意である')

F_a = np.array([f1_a, f2_a, f12_a])
F = np.array([F1, F2, F12])
result = np.array(['帰無仮説を棄却する', '帰無仮説を棄却する', '帰無仮説を棄却する'])

f_result = pd.DataFrame({'Fα':F_a,
                     'F値':F,
                     '検定結果':result})
f_result.index = ['土の種類', '肥料の量', '交互作用']
f_result

2元配置分散分析を次のようにモデル化する
Yijk = µ + αi + βj + γij + εijk
H_0: γij = 0 がすべてのi,jについて成り立つ
H_1: γij ≠ 0 となるiが存在する
α = 0.05
4.142316647264262 3.238871517453585
帰無仮説を棄却する
交互作用は有意である
H_0: αi  = 0 がすべてのi,jについて成り立つ
H_1: αi  ≠ 0 となるiが存在する
α = 0.05
46.333236321303914 4.493998477666352
帰無仮説を棄却する
土の種類は有意である
H_0: βj  = 0 がすべてのi,jについて成り立つ
H_1: βj  ≠ 0 となるiが存在する
α = 0.05
46.333236321303914 3.238871517453585
帰無仮説を棄却する
肥料の量は有意である


,Fα,F値,検定結果
土の種類,4.493998,46.333236,帰無仮説を棄却する
肥料の量,3.238872,29.484963,帰無仮説を棄却する
交互作用,3.238872,4.142317,帰無仮説を棄却する


In [ ]:
print('交互作用が有意なので、µij_hatの不偏推定値はyij_meanとなる')
print(yij_mean)
print('したがって、最適水準は、土Bかつ400gである。')
tr_a = sp.stats.t.ppf(0.975, dfr)
delta = tr_a * np.sqrt(Vr/3)
print('µij_hatの信頼限界： µij_hat ± {:.2f}'.format(delta))

交互作用が有意なので、µij_hatの不偏推定値はyij_meanとなる
         100        200        300        400
A  14.566667  15.866667  17.333333  18.500000
B  16.333333  18.033333  20.533333  24.666667
したがって、最適水準は、土Bかつ400gである。
µij_hatの信頼限界： µij_hat ± 1.46


In [ ]:
yij_mean

,100,200,300,400
A,14.566667,15.866667,17.333333,18.500000
B,16.333333,18.033333,20.533333,24.666667
